In [1]:
!pip install jsonlines openai==0.28 transformers accelerate tensorflow==2.14 nest_asyncio

  Using cached jsonlines-4.0.0-py3-none-any.whl (8.7 kB)
  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Using cached transformers-4.36.0-py3-none-any.whl (8.2 MB)
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


ERROR: Could not find a version that satisfies the requirement tensorflow==2.14 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1)
ERROR: No matching distribution found for tensorflow==2.14
You should consider upgrading via the 'c:\users\harrithha\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import argparse
import tqdm
import os
import re
import tqdm
import time
import requests
import json
import nest_asyncio
import openai
import jsonlines
from typing import List, Dict, Any
import asyncio
import sys
from transformers import AutoTokenizer
import transformers
import torch

c:\Users\Harrithha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
key = ""
fin = "qapairs.json"
prompt = '''
You are an AI assistant to answer questions. You are provided set of tools (indeed functions), description for each of the tool, arguments that each tool can take, description for each of these arguments, type of each of these arguments in the below text. Given a query, Your task involves indetifying the required set of tools to solve the query and listng them out in a json format in the order (should match with the order in which these tools should be called to solve the query) along with the list out the arguments with their values needed for each tool.

Following is the tool and it's description as reference to figure out which api functions are needed to solve the query:
{
    "tool_description": "Handle the routine tasks associated with a product's lifecycle and reduce the need for manual intervention.",
    "tool_name": "Agent007",
    "title": "Agent007",
    "api_list": [
        {
            "name": "works_list",
            "url": "",
            "description": "Returns a list of work items matching the request.",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "applies_to_part",
                    "argument_description": "Filters for work belonging to any of the provided parts",
                    "argument_type": "array of strings",
                    "argument_value_example": "['FEAT-123', 'ENH-123', 'PROD-123', 'CAPL-123']"
                },
                {
                    "argument_name": "created_by",
                    "argument_description": "Filters for work created by any of these users",
                    "argument_type": "array of strings",
                    "argument_value_example": "['DEVU-123']"
                },
                {
                    "argument_name": "issue.priority",
                    "argument_description": "Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2, p3",
                    "argument_type": "array of strings"
                },
                {
                    "argument_name": "issue.rev_orgs",
                    "argument_description": "Filters for issues with any of the provided Rev organizations",
                    "argument_type": "array of strings",
                    "argument_value_example": "['REV-123']"
                },
                {
                    "argument_name": "limit",
                    "argument_description": "The maximum number of works to return. The default is '50'",
                    "argument_type": "integer (int32)"
                },
                {
                    "argument_name": "owned_by",
                    "argument_description": "Filters for work owned by any of these users",
                    "argument_type": "array of strings",
                    "argument_value_example": "['DEVU-123']"
                },
                {
                    "argument_name": "stage.name",
                    "argument_description": "Filters for records in the provided stage(s) by name",
                    "argument_type": "array of strings"
                },
                {
                    "argument_name": "ticket.needs_response",
                    "argument_description": "Filters for tickets that need a response",
                    "argument_type": "boolean"
                },
                {
                    "argument_name": "ticket.rev_org",
                    "argument_description": "Filters for tickets associated with any of the provided Rev organizations",
                    "argument_type": "array of strings",
                    "argument_value_example": "['REV-123']"
                },
                {
                    "argument_name": "ticket.severity",
                    "argument_description": "Filters for tickets with any of the provided severities. Allowed values: blocker, high, low, medium",
                    "argument_type": "array of strings"
                },
                {
                    "argument_name": "ticket.source_channel",
                    "argument_description": "Filters for tickets with any of the provided source channels",
                    "argument_type": "array of strings"
                },
                {
                    "argument_name": "type",
                    "argument_description": "Filters for work of the provided types. Allowed values: issue, ticket, task",
                    "argument_type": "array of strings"
                }
            ]
        },
        {
            "name": "summarize_objects",
            "url": "",
            "description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal implementation detail.",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "objects",
                    "argument_description": "List of objects to summarize",
                    "argument_type": "array of objects"
                }
            ]
        },
        {
            "name": "prioritize_objects",
            "url": "",
            "description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail.",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "objects",
                    "argument_description": "A list of objects to be prioritized",
                    "argument_type": "array of objects"
                }
            ]
        },
        {
            "name": "add_work_items_to_sprint",
            "url": "",
            "description": "Adds the given work items to the sprint",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "work_ids",
                    "argument_description": "A list of work item IDs to be added to the sprint.",
                    "argument_type": "array of strings"
                },
                {
                    "argument_name": "sprint_id",
                    "argument_description": "The ID of the sprint to which the work items should be added",
                    "argument_type": "str"
                }
            ]
        },
        {
            "name": "get_sprint_id",
            "url": "",
            "description": "Returns the ID of the current sprint",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": []
        },
        {
            "name": "get_similar_work_items",
            "url": "",
            "description": "Returns a list of work items that are similar to the given work item",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "work_id",
                    "argument_description": "The ID of the work item for which you want to find similar items",
                    "argument_type": "string"
                }
            ]
        },
        {
            "name": "search_object_by_name",
            "url": "",
            "description": "Given a search string, returns the id of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest.",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "query",
                    "argument_description": "The search string, could be for example customer’s name, part name, user name.",
                    "argument_type": "string"
                }
            ]
        },
        {
            "name": "create_actionable_tasks_from_text",
            "url": "",
            "description": "Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item.",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": [
                {
                    "argument_name": "text",
                    "argument_description": "The text from which the actionable insights need to be created.",
                    "argument_type": "string"
                }
            ]
        },
        {
            "name": "who_am_i",
            "url": "",
            "description": "Returns the ID of the current user",
            "method": "GET",
            "required_parameters": [],
            "optional_parameters": []
        }
    ],
    "standardized_name": "Agent007"
}

Here are a few examples of the output that we expect from you for the given tool (follow the same format as mentioned in the output):

Input:
        {
            "query": "Summarize issues similar to don:core:dvrv-us-1:devo/0:issue/1",
            "query_id": 1
        }
Output:
        {"answer": [
            {
                "tool_name": "get_similar_work_items",
                "arguments": [
                    {
                        "argument_name": "work_id",
                        "argument_value": "don:core:dvrv-us-1:devo/0:issue/1"
                    }
                ]
            },
            {
                "tool_name": "summarize_objects",
                "arguments": [
                    {
                        "argument_name": "objects",
                        "argument_value": "$$PREV[0]"
                    }
                ]
            }
        ]
        }

If you do not know the answer, please guess a most probable answer. Only include the answer in your response. Do not explain.
Convert the response to a json file.
'''
fout = 'out.json'

In [4]:
nest_asyncio.apply()

class OpenAIgpt():

    def __init__(self):
        return

    def clean_str(self, string):
        pattern = re.compile(r'^\d+\. ', flags=re.MULTILINE)
        string = pattern.sub('', string)
        return string.strip()

    async def dispatch_openai_requests(
        self,
        messages_list: List[List[Dict[str, Any]]],
        model: str,
        temperature: float,
        max_tokens: int,
        top_p: float,
    ) -> List[str]:
        """Dispatches requests to OpenAI API asynchronously.

            Args:
                messages_list: List of messages to be sent to OpenAI ChatCompletion API.
                model: OpenAI model to use.
                temperature: Temperature to use for the model.
                max_tokens: Maximum number of tokens to generate.
                top_p: Top p to use for the model.
            Returns:
                List of responses from OpenAI API.
            """
        async_responses = [
            openai.ChatCompletion.acreate(
                model=model,
                messages=x,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p,
            )
            for x in messages_list
        ]
        return await asyncio.gather(*async_responses)

    async def main(self, key, file, prompt, fout):
        openai.api_key = key
        file_path = file
        with open(file_path, 'r') as f:
            contents = json.load(f)

        query_messages = []
        attributes = []
        for item in contents:
            message = item["question"]
            query_messages.append([
                {"role": "system", "content": prompt},
                {"role": "user", "content": message}
            ])
        generated_text = []
        try:
            total_time = 0.0
            for i in tqdm.trange(0, len(query_messages), 5):
                start_time = time.time()
                response = await self.dispatch_openai_requests(
                    messages_list=query_messages[i:i + 5],
                    model="gpt-3.5-turbo",
                    temperature=1.0,
                    max_tokens=2048,
                    top_p=1.0,
                )
                for j in range(len(response)):
                    generated_text.append(
                        {response[j]["choices"][0]["message"]["content"]})
                end_time = time.time()  # Measure end time for each instance
                instance_time = end_time - start_time
                print(f"Instance {i//5 + 1} Time: {instance_time} seconds")
                total_time += instance_time
            average_time = total_time / 2
            print(average_time)
        except Exception as e:
            print(f"Error: {e}")
        return generated_text

# Assuming you have defined key, fin, prompt, and fout
ob = OpenAIgpt()
ans = asyncio.run(ob.main(key, fin, prompt, fout))

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:14<00:14, 14.92s/it]

Instance 1 Time: 14.922280550003052 seconds


100%|██████████| 2/2 [00:27<00:00, 13.66s/it]

Instance 2 Time: 12.3860764503479 seconds
13.654178500175476


In [6]:
print(ans)

[{'{"answer": [\n    {\n        "tool_name": "get_similar_work_items",\n        "arguments": [\n            {\n                "argument_name": "work_id",\n                "argument_value": "don:core:dvrv-us-1:devo/0:issue/1"\n            }\n        ]\n    },\n    {\n        "tool_name": "summarize_objects",\n        "arguments": [\n            {\n                "argument_name": "objects",\n                "argument_value": "$$PREV[0]"\n            }\n        ]\n    }\n]\n}'}, {'{"answer": [\n    {\n        "tool_name": "Agent007",\n        "arguments": [\n            {\n                "argument_name": "text",\n                "argument_value": "What is the meaning of life?"\n            }\n        ]\n    }\n]}'}, {'{"answer": [\n    {\n        "tool_name": "works_list",\n        "arguments": [\n            {\n                "argument_name": "issue.priority",\n                "argument_value": "[\'p0\']"\n            }\n        ]\n    },\n    {\n        "tool_name": "prioritize_obje